In [13]:
import pandas as pd
import re

# пересоберем файл после ручных правок
grouped_df7 =  pd.read_csv('grouped_rev_7.csv', sep=';')
result_df8 = grouped_df7.groupby(['author', 'title'])['uid'].agg(lambda x: ', '.join(map(str, x))).reset_index()
result_df8.rename(columns={'group': 'result'}, inplace=True)
result_df8.to_csv('grouped_8.csv', index = False)
result_df8

,author,title,uid
0,Абашидзе И. В.,₽Произведения по выбору,"1_7565, 1_7830, 1_8112, 1_8393, 1_8689, 1_8954..."
1,Абгарян Н. Ю.,Произведения по выбору,"3_152, 3_153, 3_154, 3_155, 3_156"
2,Абрамов Ф. А.,Безотцовщина,2_13077
3,Абрамов Ф. А.,Братья и сестры,"3_157, 3_158, 3_159, 3_160"
4,Абрамов Ф. А.,Две зимы и три лета,"1_5712, 1_5976, 1_6240, 1_6504, 1_6768, 1_7032..."
...,...,...,...
3054,₽Неизвестный автор,Донские рассказы,2_11780
3055,₽Пушкин А. С.,₽Смерть поэта,1_10149
3056,₽Саути Р.,Суд Божий над Епископом,2_12193
3057,₽Скотт В.,"Замок Смальгольм, или Иванов вечер",2_12207


In [14]:
# развернем в линии 
df_expanded = result_df8.assign(uid=result_df8['uid'].str.split(', ')).explode('uid')
df_expanded.reset_index(drop=True, inplace=True)
df_expanded

,author,title,uid
0,Абашидзе И. В.,₽Произведения по выбору,1_7565
1,Абашидзе И. В.,₽Произведения по выбору,1_7830
2,Абашидзе И. В.,₽Произведения по выбору,1_8112
3,Абашидзе И. В.,₽Произведения по выбору,1_8393
4,Абашидзе И. В.,₽Произведения по выбору,1_8689
...,...,...,...
18138,₽Неизвестный автор,Донские рассказы,2_11780
18139,₽Пушкин А. С.,₽Смерть поэта,1_10149
18140,₽Саути Р.,Суд Божий над Епископом,2_12193
18141,₽Скотт В.,"Замок Смальгольм, или Иванов вечер",2_12207


In [15]:
# прочитаем все исходные файлы
df1 = pd.read_csv('1.tsv', sep='\t')  # 1) Советский период
df2 = pd.read_csv('2.tsv', sep='\t')  # 2) 1990-е годы
df3 = pd.read_csv('3.tab', sep='\t')  # 3) Постсоветский период 
concatenated_df = pd.read_csv('concatenated.csv', header =0)

# включим уникальный номер для каждого датасета для облегчения иденттификации
df1['uid']='1_'+df1.index.astype(str)
df2['uid']='2_'+df2.index.astype(str)
df3['uid']='3_'+df3.index.astype(str)

# добавим префиксы колонок 
df1.columns = ['1_author', '1_title', '1_comment', '1_curriculum', '1_id', '1_year', '1_grade','1_priority', 'uid']
df2.columns = ['2_author (автор)', '2_title (название)', '2_comment (раздел)', '2_curriculum','2_id', '2_year (год УП)', '2_grade (класс)','2_priority (обязательное или нет)', '2_first publication', '2_quantity', '2_summary (примечание)', 'uid']
df3.columns = ['3_source_id', '3_source_year', '3_group', '3_century', '3_author', '3_title','3_genre', '3_type', '3_oblig_author', '3_oblig_title', '3_grade', '3_section','3_number_authors', '3_number_titles', '3_profound', '3_source_issue',   '3_title_cycle', '3_notes', 'uid']
concatenated_df.columns = ['uid','con_author','con_title','con_grade','con_year','con_oblig']

# добавим каждое исходное поле по uid
df_expanded = df_expanded.merge(df1, on='uid', how='left')
df_expanded = df_expanded.merge(df2, on='uid', how='left')
df_expanded = df_expanded.merge(df2, on='uid', how='left')
df_expanded = df_expanded.merge(concatenated_df, on='uid', how='left')

df_expanded.to_csv('expanded.csv', index = False)
df_expanded

,author,title,uid,1_author,1_title,1_comment,1_curriculum,1_id,1_year,1_grade,...,2_grade (класс)_y,2_priority (обязательное или нет)_y,2_first publication_y,2_quantity_y,2_summary (примечание)_y,con_author,con_title,con_grade,con_year,con_oblig
0,Абашидзе И. В.,₽Произведения по выбору,1_7565,Абашидзе И.В.,NaN,NaN,81-82 ПВСШ,42.0,1981,10.0,...,NaN,NaN,NaN,NaN,NaN,Абашидзе И.В.,NaN,10,1981.0,0.0
1,Абашидзе И. В.,₽Произведения по выбору,1_7830,Абашидзе И.В.,NaN,NaN,82-83 ПВСШ,43.0,1982,10.0,...,NaN,NaN,NaN,NaN,NaN,Абашидзе И.В.,NaN,10,1982.0,0.0
2,Абашидзе И. В.,₽Произведения по выбору,1_8112,Абашидзе И.В.,NaN,NaN,83-84 ПВСШ,44.0,1983,10.0,...,NaN,NaN,NaN,NaN,NaN,Абашидзе И.В.,NaN,10,1983.0,0.0
3,Абашидзе И. В.,₽Произведения по выбору,1_8393,Абашидзе И.В.,NaN,NaN,84-85 ПВСШ,45.0,1984,10.0,...,NaN,NaN,NaN,NaN,NaN,Абашидзе И.В.,NaN,10,1984.0,0.0
4,Абашидзе И. В.,₽Произведения по выбору,1_8689,Абашидзе И.В.,NaN,NaN,85 ПВСШ,46.0,1985,10.0,...,NaN,NaN,NaN,NaN,NaN,Абашидзе И.В.,NaN,10,1985.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18138,₽Неизвестный автор,Донские рассказы,2_11780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11,самостоятельно,NaN,NaN,NaN,NaN,Донские рассказы,11,1994.0,0.0
18139,₽Пушкин А. С.,₽Смерть поэта,1_10149,Пушкин А.С.,Смерть поэта,NaN,91 ПСОУЗ,50.0,1991,9.0,...,NaN,NaN,NaN,NaN,NaN,Пушкин А.С.,Смерть поэта,9,1991.0,0.0
18140,₽Саути Р.,Суд Божий над Епископом,2_12193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,NaN,NaN,не найден,NaN,Саути Р.,Суд Божий над Епископом,6,1995.0,0.0
18141,₽Скотт В.,"Замок Смальгольм, или Иванов вечер",2_12207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,самостоятельно,NaN,не найден,NaN,Скотт В.,"Замок Смальгольм, или Иванов вечер",6,1995.0,0.0
